### Basic example of provision calculation when demands in buildings are calculated and the adjacency matrix is computed.
#### Threshold - the value above which buildings will be marked as not meeting the specified value.

In [ ]:

from objectnat import get_service_provision
import geopandas as gpd
import pandas as pd

# Importing necessary libraries:
# - 'get_service_provision' calculates service provision based on adjacency matrix
# - 'geopandas' for handling geospatial data
# - 'pandas' for working with tabular data

builds = gpd.read_parquet("examples_data/buildings.parquet").to_crs(32636)
services = gpd.read_parquet("examples_data/services.parquet").to_crs(32636)
adjacency_matrix = pd.read_parquet('examples_data/matrix_time.parquet')
# Reading building and service data, reprojecting them to EPSG:32636 (UTM zone), and loading the precomputed adjacency matrix

adjacency_matrix.index = adjacency_matrix.index.astype(int)
services.index = services.index.astype(int)
builds.index = builds.index.astype(int)
adjacency_matrix.columns = adjacency_matrix.columns.astype(int)
# Ensuring the indices and columns of the adjacency matrix and GeoDataFrames are integers for proper matching

build_prov, services_prov, links_prov = get_service_provision(
    buildings=builds,
    services=services,
    adjacency_matrix=adjacency_matrix,
    threshold=10,
)
# Calculating service provision for each building. The 'threshold' parameter (10) defines the maximum allowable distance (or time)
# beyond which buildings are considered to not meet the required service provision.


# Visualize data
m1 = build_prov.reset_index().explore(column='avg_dist', cmap='RdYlGn_r', tiles='CartoDB positron')
# Visualizing buildings provision on a map using 'explore'. 
# The 'avg_dist' column (average distance to services) is visualized with a color map 'RdYlGn_r'

links_prov.explore(m=m1, column='service_index', cmap='prism', style_kwds={'opacity': 0.5})
# Visualizing links (connections between buildings and services) on the same map.
# 'service_index' shows which service each link corresponds to.

services_prov.explore(m=m1, color='red')
# Visualizing service locations on the same map, using red markers for easy distinction.

### To recalculate demands, capacity, and links according to max distance value
#### This section recalculates the service provision with a new threshold.

In [ ]:
from objectnat import recalculate_links

vmax = build_prov['avg_dist'].max()
# Extracting the maximum average distance for later visualization use.

build_prov2, services_prov2, links_prov2 = recalculate_links(build_prov, services_prov, links_prov, 15)
# Recalculating the service provision based on a new threshold of 15 (e.g., kilometers or minutes).
# This adjusts the allocation of services to buildings based on this new maximum allowed distance.

# Visualize data
m1 = build_prov2.reset_index().explore(column='avg_dist', cmap='RdYlGn_r', tiles='CartoDB positron', vmax=vmax)
# 'vmax' ensures the same color scaling is used for comparison with previous maps.
links_prov2.explore(m=m1, column='service_index', cmap='prism', style_kwds={'opacity': 0.5})
services_prov2.explore(m=m1, color='red')

### To clip provision
#### This section demonstrates how to clip (limit) the provision analysis to a subset of buildings based on a geographic region.


In [ ]:
from objectnat import clip_provision

vmax = build_prov['avg_dist'].max()
# Extracting the maximum average distance again for consistent visualization.

to_clip_gdf = builds.iloc[500:503].copy()
to_clip_gdf['geometry'] = to_clip_gdf['geometry'].buffer(500)
# Selecting a small subset of buildings (500 to 503) and buffering their geometries by 500 units to create a clipping area.
# This buffer defines the region around these buildings where we want to limit the service provision analysis.

build_prov_clipped, services_prov_clipped, links_prov_clipped = clip_provision(build_prov2, services_prov2, links_prov2,
                                                                               to_clip_gdf)
# Clipping the service provision analysis to the area defined by 'to_clip_gdf'. This limits the results to buildings and services
# that fall within the specified region.

# Visualize data
m1 = build_prov_clipped.reset_index().explore(column='avg_dist', cmap='RdYlGn_r', tiles='CartoDB positron', vmax=vmax)
# Visualizing the clipped building provision data with the same color scaling.
links_prov_clipped.explore(m=m1, column='service_index', cmap='prism', style_kwds={'opacity': 0.5})
services_prov_clipped.explore(m=m1, color='red')